In [6]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
import time
geolocator = Nominatim(user_agent="OLABikes")

In [10]:
df_august = pd.read_parquet('/home/misango/code/TaxiRequest_ML_Forecaster/Taxi_Data/parquet/yellow_tripdata_2023-08.parquet')
df_september = pd.read_parquet('/home/misango/code/TaxiRequest_ML_Forecaster/Taxi_Data/parquet/yellow_tripdata_2023-09.parquet')
df_october = pd.read_parquet('/home/misango/code/TaxiRequest_ML_Forecaster/Taxi_Data/parquet/yellow_tripdata_2023-10.parquet')
df_november = pd.read_parquet('/home/misango/code/TaxiRequest_ML_Forecaster/Taxi_Data/parquet/yellow_tripdata_2023-11.parquet')

df_august.to_csv('/home/misango/code/TaxiRequest_ML_Forecaster/Taxi_Data/csv/nyc_aug_data.csv')
df_september.to_csv('/home/misango/code/TaxiRequest_ML_Forecaster/Taxi_Data/csv/nyc_sep_data.csv')
df_october.to_csv('/home/misango/code/TaxiRequest_ML_Forecaster/Taxi_Data/csv/nyc_oct_data.csv')
df_november.to_csv('/home/misango/code/TaxiRequest_ML_Forecaster/Taxi_Data/csv/nyc_nov_data.csv')

In [15]:
df_november.head(10)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-11-01 00:03:03,2023-11-01 01:04:08,2.0,13.60,1.0,N,132,26,2,61.8,2.75,0.5,0.00,0.00,1.0,66.05,0.0,1.75
1,1,2023-11-01 00:03:28,2023-11-01 00:23:59,0.0,3.50,1.0,N,140,7,1,20.5,3.50,0.5,5.10,0.00,1.0,30.60,2.5,0.00
2,2,2023-10-31 23:58:05,2023-11-01 00:54:03,4.0,18.61,2.0,N,132,230,1,70.0,0.00,0.5,16.54,6.94,1.0,99.23,2.5,1.75
3,2,2023-11-01 00:03:50,2023-11-01 00:04:59,1.0,0.39,1.0,N,236,236,1,4.4,1.00,0.5,1.88,0.00,1.0,11.28,2.5,0.00
4,2,2023-11-01 00:06:30,2023-11-01 00:14:25,1.0,1.20,1.0,N,236,141,1,10.0,1.00,0.5,3.00,0.00,1.0,18.00,2.5,0.00
5,2,2023-11-01 00:17:18,2023-11-01 00:23:39,3.0,0.60,1.0,N,141,263,1,7.9,1.00,0.5,2.58,0.00,1.0,15.48,2.5,0.00
6,2,2023-11-01 00:14:49,2023-11-01 00:39:44,1.0,5.44,1.0,N,114,62,2,28.2,1.00,0.5,0.00,0.00,1.0,33.20,2.5,0.00
7,1,2023-11-01 00:07:29,2023-11-01 00:11:01,2.0,0.90,1.0,N,48,246,1,6.5,3.50,0.5,2.30,0.00,1.0,13.80,2.5,0.00
8,2,2023-11-01 00:39:20,2023-11-01 01:00:24,1.0,6.20,1.0,N,170,42,1,27.5,1.00,0.5,8.12,0.00,1.0,40.62,2.5,0.00
9,1,2023-11-01 00:21:12,2023-11-01 00:26:45,2.0,0.40,1.0,N,144,148,2,5.8,3.50,0.5,0.00,0.00,1.0,10.80,2.5,0.00


In [18]:
# Narrowed Data Pool and trimmed Columns for the relevant data
base_path = "Taxi_Data/csv/"

columns = ['tpep_pickup_datetime',
           'tpep_dropoff_datetime',
           'trip_distance',
           'PULocationID',
           'DOLocationID',
           'total_amount']

df_2023_8 = pd.read_csv(f'{base_path}nyc_aug_data.csv', usecols=columns, nrows=1000000)
df_2023_9 = pd.read_csv(f'{base_path}nyc_sep_data.csv', usecols=columns, nrows=1000000)
df_2023_10 = pd.read_csv(f'{base_path}nyc_oct_data.csv', usecols=columns, nrows=1000000)
df_2023_11 = pd.read_csv(f'{base_path}nyc_nov_data.csv', usecols=columns, nrows=1000000)

print(type(df_2023_8))
print(type(df_2023_9))
print(type(df_2023_10))
print(type(df_2023_11))

# Concatenate DataFrames
df_2023_q4 = pd.concat([df_2023_8, df_2023_9, df_2023_10, df_2023_11])

# Debugging: Print length of concatenated DataFrame
print("Length of concatenated DataFrame:", df_2023_q4.shape[0])


<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
Length of concatenated DataFrame: 4000000


In [22]:
df_2023_q4.head(10)

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID,total_amount
0,2023-08-01 00:26:44,2023-08-01 00:45:25,4.30,263,90,32.25
1,2023-08-01 00:55:42,2023-08-01 01:00:53,0.00,132,132,9.35
2,2023-08-01 00:32:04,2023-08-01 01:09:03,20.61,132,13,90.55
3,2023-08-01 00:13:37,2023-08-01 00:41:15,13.37,138,254,69.90
4,2023-08-01 00:24:46,2023-08-01 00:41:31,3.20,140,143,21.00
5,2023-08-01 00:44:01,2023-08-01 00:44:32,0.00,143,143,19.00
6,2023-08-01 00:03:46,2023-08-01 00:09:37,1.20,50,142,15.50
7,2023-08-01 00:10:19,2023-08-01 00:23:11,2.40,142,41,20.50
8,2023-08-01 00:48:55,2023-08-01 00:56:00,1.20,230,229,17.30
9,2023-08-01 00:11:29,2023-08-01 00:43:06,11.05,132,37,60.43


In [24]:
df_2023_q4.shape


(4000000, 6)

In [23]:
#Cleaning the NA values
def clean_data(df_2023_q4, test=False, predict=False):
    df_2023_q4 = df_2023_q4.dropna(how='any', axis='rows')
    df_2023_q4 = df_2023_q4[(df_2023_q4.PULocationID != 0) | (df_2023_q4.DOLocationID!= 0)]
    df_2023_q4 = df_2023_q4[(df_2023_q4.PULocationID != 0) | (df_2023_q4.PULocationID != 0)]
    
    if "total_amount" in list(df_2023_q4):
        df_2023_q4 = df_2023_q4[df_2023_q4.total_amount.between(5, 45)]
    
    return df_2023_q4
df_2023_q4_clean = clean_data(df_2023_q4)
df_2023_q4_clean.shape

(3374067, 6)

In [25]:
# drop rows with coordinates outside NYC based on Data from https://data.cityofnewyork.us/Transportation/NYC-Taxi-Zones/d3c5-ddgc
# The Max ID limit is 263 hence all values with ID greater than 263 are removed
#Work on the drop-off biases for future cleaning
 
def clean_coordinates(df_2023_q4_clean):
    nrows = df_2023_q4_clean.shape[0]
    df_2023_q4_clean.drop(df_2023_q4_clean.index[

            ~((df_2023_q4_clean['PULocationID'].between(0, 263)) &
              (df_2023_q4_clean['DOLocationID'].between(0, 263))) 
        
    ], inplace=True)

    print("Number of rows removed due to wrong coordinates is {}".format(nrows - df_2023_q4_clean.shape[0]))
    
clean_coordinates(df_2023_q4_clean)

Number of rows removed due to wrong coordinates is 142110


In [32]:
df_2023_q4_clean.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID,total_amount
0,2023-08-01 00:26:44,2023-08-01 00:45:25,4.3,263,90,32.25
1,2023-08-01 00:55:42,2023-08-01 01:00:53,0.0,132,132,9.35
4,2023-08-01 00:24:46,2023-08-01 00:41:31,3.2,140,143,21.00
5,2023-08-01 00:44:01,2023-08-01 00:44:32,0.0,143,143,19.00
6,2023-08-01 00:03:46,2023-08-01 00:09:37,1.2,50,142,15.50


In [33]:
def clean_trip_duration(df):
    # convert from object to datetime
    df_2023_q4_clean['tpep_pickup_datetime']  = pd.to_datetime(df['tpep_pickup_datetime'])
    df_2023_q4_clean['tpep_dropoff_datetime']  = pd.to_datetime(df['tpep_dropoff_datetime'])
    
    # copute the time diffrance between pickup & dropoff
    # to covert from nanosecondes to minutes we devide by 1000000000 then by 60
    # store trip_duratin column
    trip_duration = np.array(df['tpep_dropoff_datetime']-df_2023_q4_clean['tpep_pickup_datetime'])
    trip_duration = trip_duration/1000000000/60
    df_2023_q4_clean['trip_duration'] = trip_duration.astype(float)
    
    # drop all records that have trip_duration > 2 hours
    #                            trip_duration <= 0
    #                            trip_distance <= 0
    nrows = df_2023_q4_clean.shape[0]
    df_2023_q4_clean.drop(df_2023_q4_clean[(df['trip_duration'] > 160) | 
               (df_2023_q4_clean['trip_duration'] <= 0)].index, inplace = True)
    print("Number of rows removed due to wrong trip_duration {}".format(nrows - df_2023_q4_clean.shape[0]))
    
    
clean_trip_duration(df_2023_q4_clean)

Number of rows removed due to wrong trip_duration 77535


In [34]:
def clean_pickuptime(df_2023_q4_clean):
    return df_2023_q4_clean.rename(columns={'tpep_pickup_datetime': 'pickup_time'})
def clean_dropofftime(df_2023_q4_clean):
    return df_2023_q4_clean.rename(columns={'tpep_dropoff_datetime': 'dropoff_time'})

df_2023_q4_clean_pickup = clean_pickuptime(df_2023_q4_clean)
df_2023_q4_clean = clean_dropofftime(df_2023_q4_clean_pickup)

In [40]:
df_2023_q4_clean.head(10)

,pickup_time,dropoff_time,trip_distance,PULocationID,DOLocationID,total_amount,trip_duration
4,2023-08-01 00:24:46,2023-08-01 00:41:31,3.20,140,143,21.00,16.0
6,2023-08-01 00:03:46,2023-08-01 00:09:37,1.20,50,142,15.50,5.0
7,2023-08-01 00:10:19,2023-08-01 00:23:11,2.40,142,41,20.50,12.0
8,2023-08-01 00:48:55,2023-08-01 00:56:00,1.20,230,229,17.30,7.0
10,2023-08-01 00:29:31,2023-08-01 00:56:19,9.70,132,177,41.20,26.0
11,2023-08-01 00:38:43,2023-08-01 00:47:14,1.80,230,162,15.70,8.0
14,2023-08-01 00:04:32,2023-08-01 00:15:25,3.31,87,256,25.56,10.0
15,2023-08-01 00:46:39,2023-08-01 01:05:32,5.18,249,263,34.80,18.0
18,2023-08-01 00:01:31,2023-08-01 00:06:27,1.86,68,50,16.30,4.0
19,2023-08-01 00:08:00,2023-08-01 00:28:16,4.43,50,148,35.38,20.0


In [41]:
def clean_trip_distance(df_2023_q4_clean):
    nrows = df_2023_q4_clean.shape[0]
    df_2023_q4_clean.drop(df_2023_q4_clean[(df_2023_q4_clean['trip_distance'] <= 0) | (df_2023_q4_clean['trip_distance'] > 77.5)].index, inplace = True)
    print("Number of rows removed due to speed outliers {}".format(nrows - df_2023_q4_clean.shape[0]))
    
clean_trip_distance(df_2023_q4_clean)


Number of rows removed due to speed outliers 44221


In [42]:
def compute_speed(df_2023_q4_clean):
    # computing Taxi speed average (mile/hour)
    df_2023_q4_clean['speed'] = df_2023_q4_clean['trip_distance']/df_2023_q4_clean['trip_duration']*60
    
def clean_speed(df_2023_q4_clean):

    # Removing speed anomaly/outliers
    nrows = df_2023_q4_clean.shape[0]
    df_2023_q4_clean.drop(df_2023_q4_clean[((df_2023_q4_clean['speed'] <= 0) | (df_2023_q4_clean['speed'] > 63.0))].index, inplace = True)
    print("Number of rows removed due to speed outliers {}".format(nrows - df_2023_q4_clean.shape[0]))


compute_speed(df_2023_q4_clean)

In [43]:
clean_speed(df_2023_q4_clean)

Number of rows removed due to speed outliers 460


In [44]:
df_2023_q4_clean.to_csv('/home/misango/code/TaxiRequest_ML_Forecaster/Taxi_Data/csv/final_nyc_q4_clean.csv',index = False, compression = 'gzip')

In [45]:
df_2023_q4_clean

,pickup_time,dropoff_time,trip_distance,PULocationID,DOLocationID,total_amount,trip_duration,speed
4,2023-08-01 00:24:46,2023-08-01 00:41:31,3.20,140,143,21.00,16.0,12.000000
6,2023-08-01 00:03:46,2023-08-01 00:09:37,1.20,50,142,15.50,5.0,14.400000
7,2023-08-01 00:10:19,2023-08-01 00:23:11,2.40,142,41,20.50,12.0,12.000000
8,2023-08-01 00:48:55,2023-08-01 00:56:00,1.20,230,229,17.30,7.0,10.285714
10,2023-08-01 00:29:31,2023-08-01 00:56:19,9.70,132,177,41.20,26.0,22.384615
...,...,...,...,...,...,...,...,...
999995,2023-11-09 19:48:52,2023-11-09 20:00:00,1.67,261,249,20.46,11.0,9.109091
999996,2023-11-09 19:27:34,2023-11-09 19:34:40,1.40,237,236,18.95,7.0,12.000000
999997,2023-11-09 19:50:45,2023-11-09 20:18:22,4.60,237,68,41.60,27.0,10.222222
999998,2023-11-09 19:21:42,2023-11-09 19:45:33,8.20,132,86,41.65,23.0,21.391304
